In [ ]:

# #http://ix.io/3rxn
# t2 = r'''
# id,PredictionString
# abe007e3fc08_image,None 1 0 0 1 1
# b92e6f1b6c1b_image,opacity 0.448915 36 100 37 169 opacity 0.514129 146 204 55 183
# 948e9f366810_image,opacity 0.600151 68 136 78 222 opacity 0.615949 179 253 67 211
# 3cccbd897c39_image,None 1 0 0 1 1
# 3bb8935d30d9_image,None 1 0 0 1 1
# 37d6750b5282_image,opacity 0.289108 52 108 85 197 opacity 0.450337 153 209 87 187
# 67f192ad1a53_image,opacity 0.42714 147 199 50 164 opacity 0.579659 45 105 36 140
# 7f090fbd8e7c_image,None 1 0 0 1 1
# 898d24f3cda6_image,opacity 0.479425 17 105 35 171
# a0f6e42396fc_image,opacity 0.37193 171 231 106 182 opacity 0.431259 67 127 61 179
# 422c0e145066_image,None 1 0 0 1 1
# 46719b856de1_image,None 1 0 0 1 1
# 31c07523a69a_image,opacity 0.427264 53 113 78 200 opacity 0.565582 168 238 58 194
# f77d7d1aebab_image,None 1 0 0 1 1
# ccc5b63ca96d_image,None 1 0 0 1 1
# 5e8ac1fe2b82_image,opacity 0.318599 166 232 126 242 opacity 0.427661 48 120 78 234
# '''


In [ ]:
# import io
# import pandas
# import numpy
# with io.StringIO(t2) as f:
#     t3 = pandas.read_csv(f)
# t3.PredictionString = t3.PredictionString.apply(lambda x: x.replace('Negative', 'negative').replace('None', 'none'))
# for i in range(t3.shape[0]):
#     t4 = t3.iloc[i].PredictionString
#     if 'opacity' in t4:
#         t5 = t4.split()
#         assert len(t5) % 6 == 0, 'error'
#         t6 = numpy.array(t5).reshape(-1, 6)
#         assert numpy.all(t6[:, 0] == 'opacity'), 'error 2'
#         #print(t6)
#         assert numpy.all(t6[:, 2].astype(numpy.int32) < t6[:, 3].astype(numpy.int32)), 'error 3'
#         assert numpy.all(t6[:, 4].astype(numpy.int32) < t6[:, 5].astype(numpy.int32)), 'error 4'
#         t7 = numpy.stack([
#             t6[:, 0],
#             t6[:, 1],
#             t6[:, 2], t6[:, 4],
#             t6[:, 3], t6[:, 5]
#         ]).T
#         #print(t7)
#         #print(i)
#         t8 = ' '.join(t7.flatten().tolist())
#         #print(t8)
#         t3.iloc[i].PredictionString = t8
# assert (submission_df['id'].values == t3['id'].values).all()

# display(t3)


# t3.to_csv('submission.csv', index=False)

# #https://i.imgur.com/3ARpdM0.png
# #incorrect format of labels, big letters, etc
    
# t9 = t3.copy()
# t9.PredictionString = submission_df.PredictionString
# t9.iloc[2476].values[1] = 'opacity 0.318599 166 126 232 242 opacity 0.427661 48 78 120 234'
# display(t9)
# t9.to_csv('submission.csv', index=False)

In [ ]:
import pandas as pd
from path import Path
import pprint

# def display(*args, **kwargs):
#     pprint.pprint([args, kwargs])
    
dataset_path = Path('../input/siim-covid19-detection')
submission_df = pd.read_csv(dataset_path/'sample_submission.csv')


In [ ]:
submission_df.head(20)

In [ ]:
submission_df.tail(20)

In [ ]:
submission_df.shape

In [ ]:
submission_df.to_csv('submission.csv', index=False)

### default sumbimission gets a score of 0.1 - hehe

### use info from study level classifier now

https://www.kaggle.com/rbhambri/image-clf-chexnet-vanilla-cnn

https://www.kaggle.com/awsaf49/siim-covid-19-study-level-infer

### sample

898d24f3cda6_image,opacity 0.479425 17 105 35 171


In [ ]:
image_id = '898d24f3cda6_image'

In [ ]:
submission_df[submission_df.id == image_id]

In [ ]:
! ls ../input/covid-jpg-512/test | grep cda6.jpg

In [ ]:
! ls ../input/siim-covid19-detection/test/*/* | grep 898d24f3cda6

### load model and generator

In [ ]:
img_size = 512

In [ ]:
batch_size = 16

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
image_generator_test = ImageDataGenerator()

In [ ]:
image_df_test = submission_df[submission_df.id.str.endswith('_image')]

In [ ]:
image_df_test.shape

In [ ]:
submission_df.shape

In [ ]:
image_df_test.head()

In [ ]:
study_df_test = submission_df[submission_df.id.str.endswith('_study')]

In [ ]:
study_df_test.shape

In [ ]:
image_df_test['id_image'] = image_df_test['id'].str.replace('_image', '.jpg')

In [ ]:
image_df_test.head()

In [ ]:
test_dir = '../input/covid-jpg-512/test'

In [ ]:

test_generator = image_generator_test.flow_from_dataframe(
    dataframe = image_df_test,
    directory = test_dir,
    x_col = 'id_image',
#     y_col = 'study_label',
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode=None,
#     subset='validation', 
    shuffle=False, 
#     seed=23
) 

In [ ]:
# from tensorflow.keras.applications.vgg16 import  VGG16
from tensorflow.keras.applications import DenseNet121

from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D,  BatchNormalization, Activation
# from tensorflow.keras import models

from tensorflow.keras.models import Model, Sequential

In [ ]:
def build_chextnet_model_v1():
    """
    v1 - uses densenet + chextnet weights
    """
    # load model design
    pre_model = DenseNet121(weights=None,
                        include_top=False,
                        input_shape=(img_size,img_size,3)
                        )
    out = Dense(14, activation='sigmoid')(pre_model.output)
    pre_model = Model(inputs=pre_model.input, outputs=out) 
    
    # load model wieghts
    chex_weights_path = '../input/chexnet-weights/brucechou1983_CheXNet_Keras_0.3.0_weights.h5'
    pre_model.load_weights(chex_weights_path)

    # make layers trainable?
    # pre_model.trainable = False
    pre_model.trainable = True

    # get summary/print
    pre_model.summary()
    
    
    # add future layers.
    # last_layer = pre_model.get_layer('conv5_block16_concat')
    last_layer = pre_model.layers[-2]

    print('last layer output shape: ', last_layer.output_shape)
    last_output = last_layer.output
#     last_layer

    # Flatten the output layer to 1 dimension
    # x = Flatten()(last_output)
    x = GlobalAveragePooling2D()(last_output)

    # Add a fully connected layer with 512 hidden units and ReLU activation
    # x = Dense(512, activation='relu')(x)
    # Add a dropout rate of 0.2
    # x = Dropout(0.2)(x)                  


    # # Add a fully connected layer with 128 hidden units and ReLU activation
    # x = Dense(128, activation='relu')(x)


    # Add final classification layer
    x = Dense(4, activation='softmax')(x)

    # final model
    model = Model( pre_model.input, x) 

    # model.summary()
    # plot_model(model)

    return model

In [ ]:
model = build_chextnet_model_v1()

In [ ]:
! ls ../input/image-clf-chexnet-vanilla-cnn

In [ ]:
model.load_weights('../input/image-clf-chexnet-vanilla-cnn/chextnet_model_512_july11.h5')

In [ ]:
import numpy as np

In [ ]:
# preds = np.argmax(model.predict(test_generator), axis=1)

In [ ]:
# abc

In [ ]:
# preds[:25]

In [ ]:
# preds.shape

In [ ]:
# image_df_test['pred_study_class'] = preds

In [ ]:
# image_df_test.head()

In [ ]:
label_map = {0: 'atypical', 1: 'indeterminate', 2: 'negative', 3: 'typical'}

In [ ]:
c = model.predict(test_generator)

In [ ]:
c.shape

In [ ]:
ix = 0


In [ ]:
c[ix]

In [ ]:
preds = []

for c_row in c:
#     print(c_row)
    c_row_best_ix = np.argmax(c_row)
    c_row_best = c_row[c_row_best_ix]
    
    c_row_dict = {'c_row_best_ix': c_row_best_ix, 'c_row_best': c_row_best, 'c_row': list(c_row)}
#     print(c_row_dict)
    preds.append(c_row_dict)
    
#     break

In [ ]:
# [ max(c[ix] for ix in range(1263))]

In [ ]:
preds[:10]

In [ ]:
image_df_test['predictions'] = preds

In [ ]:
image_df_test.head()

In [ ]:
def get_study_class_name(row):
    predictions = row.predictions
    
    pred_study_class = predictions['c_row_best_ix']
    
    pred_study_class_name = label_map[pred_study_class]
    return pred_study_class_name
    

In [ ]:
def get_study_class_proba(row):
    predictions = row.predictions
    c_row_best_proba = predictions['c_row_best']
    return c_row_best_proba

In [ ]:
image_df_test['study_class_name'] = image_df_test.apply(lambda row:get_study_class_name(row) , axis=1)
image_df_test['study_class_proba'] = image_df_test.apply(lambda row:get_study_class_proba(row) , axis=1)


In [ ]:
image_df_test.head()

### now link image back to study

In [ ]:
study_df_test.head()

In [ ]:
study_df_test['id'] = study_df_test['id'].str.replace('_study',"")

In [ ]:
study_df_test.head()

In [ ]:
def get_absolute_file_paths(directory):
    all_abs_file_paths = []
    for dirpath,_,filenames in os.walk(directory):
        for f in filenames:
            all_abs_file_paths.append(os.path.abspath(os.path.join(dirpath, f)))
    return all_abs_file_paths

In [ ]:
from tqdm.notebook import tqdm; tqdm.pandas();

In [ ]:
import os

In [ ]:
study_df_test["study_dir"] = "/kaggle/input/siim-covid19-detection/test/" + study_df_test["id"]
study_df_test["images_per_study"] = study_df_test.study_dir.progress_apply(lambda x: len(get_absolute_file_paths(x)))

In [ ]:
study_df_test.head()

In [ ]:
study_df_test.images_per_study.value_counts()

In [ ]:
import glob

In [ ]:
filepaths = glob.glob('/kaggle/input/siim-covid19-detection/test/**/*dcm',recursive=True)
test_df = pd.DataFrame({'filepath':filepaths,})
test_df['image_id'] = test_df.filepath.map(lambda x: x.split('/')[-1].replace('.dcm', '')+'_image')
test_df['study_id'] = test_df.filepath.map(lambda x: x.split('/')[-3].replace('.dcm', '')+'_study')
test_df.head()

In [ ]:
test_df['id'] = test_df['image_id']

In [ ]:
test_df.shape

In [ ]:
test_df.head()

In [ ]:
image_df_test.shape

In [ ]:
image_df_test.head()

In [ ]:
image_df_test2 = image_df_test.merge(test_df, 
                                     on='id',
                                 )

In [ ]:
image_df_test2.head()

In [ ]:
image_df_test2.shape

In [ ]:
image_df_test2.study_class_name.value_counts()

In [ ]:
# dict(image_df_test2.study_id.value_counts())



In [ ]:
image_df_test2.study_id.describe()


In [ ]:
image_df_test2[image_df_test2.study_id == '342624fb2b11_study']

In [ ]:
# image_df_test2[image_df_test2.study_id == 'bc73883551cf_study']
image_df_test2[image_df_test2.study_id == 'bc73883551cf_study'].iloc[0].predictions


### assign label to each study

In [ ]:
study_class_map = {}

In [ ]:
for ix, row in image_df_test2.iterrows():
#     print(ix)
#     print(row)
    
    study_id = row.study_id                                           
    study_class_name = row.study_class_name                                             
    study_class_proba = row.study_class_proba
    all_class_predictions = row.predictions['c_row']

    if study_id not in study_class_map:
        study_class_map[study_id] = (study_class_name, study_class_proba, all_class_predictions)
    
    
#     break

In [ ]:
# study_class_map

In [ ]:
submission_df.head()

In [ ]:
# def update_PredictionString(row):
# #     print(row)
#     row_id = row.id
    
#     if row_id in study_class_map:
#         study_class, study_class_proba = study_class_map[row_id]
#         study_class_proba = round(study_class_proba, 2)
#         print(row_id, study_class, study_class_proba)
    
#         PredictionString = study_class + ' ' + str(study_class_proba) + ' 0 0 1 1'
#         print(PredictionString)
#         return PredictionString
        
#     else:
#         print('warning for', row_id)
#         return row.PredictionString
    
    
#     print('===========\n')

In [ ]:
label_map = {0: 'atypical', 1: 'indeterminate', 2: 'negative', 3: 'typical'}

In [ ]:
def update_PredictionString_v2(row):
#     print(row)
    row_id = row.id
    
    if row_id in study_class_map:
        study_class, study_class_proba, all_class_predictions = study_class_map[row_id]
        study_class_proba = round(study_class_proba, 2)
#         print(row_id, study_class, study_class_proba, all_class_predictions)
    
        negative_proba = all_class_predictions[2]
        PredictionString = 'negative ' + str(negative_proba) + ' 0 0 1 1'
        
        typical_proba = all_class_predictions[3]
        PredictionString += ' typical ' + str(typical_proba) + ' 0 0 1 1'

        indeterminate_proba = all_class_predictions[1]
        PredictionString += ' indeterminate ' + str(indeterminate_proba) + ' 0 0 1 1'
        
        atypical_proba = all_class_predictions[0]
        PredictionString += ' atypical ' + str(atypical_proba) + ' 0 0 1 1'
        
        
#         print(PredictionString)
        return PredictionString
        
    else:
#         print('warning for', row_id)
        my_str = 'opacity 0.479425 17 105 35 171'
#         return row.PredictionString
        return my_str
    
    
#     print('===========\n')

In [ ]:
# def update_PredictionString_v3(row):
# #     print(row)
#     row_id = row.id
    
#     if row_id in study_class_map:
#         study_class, study_class_proba, all_class_predictions = study_class_map[row_id]
#         study_class_proba = round(study_class_proba, 2)
# #         print(row_id, study_class, study_class_proba, all_class_predictions)
    
# #         negative_proba = all_class_predictions[2]
#         negative_proba = 1
#         PredictionString = 'negative ' + str(negative_proba) + ' 0 0 1 1'
        
# #         typical_proba = all_class_predictions[3]
#         typical_proba = 1
#         PredictionString += ' typical ' + str(typical_proba) + ' 0 0 1 1'

# #         indeterminate_proba = all_class_predictions[1]
#         indeterminate_proba = 1
#         PredictionString += ' indeterminate ' + str(indeterminate_proba) + ' 0 0 1 1'
        
# #         atypical_proba = all_class_predictions[0]
#         atypical_proba = 1
#         PredictionString += ' atypical ' + str(atypical_proba) + ' 0 0 1 1'
        
        
# #         print(PredictionString)
#         return PredictionString
        
#     else:
#         print('warning for', row_id)
#         return row.PredictionString
    
    
# #     print('===========\n')

In [ ]:
# abc

In [ ]:
# submission_df.head().apply(lambda row: update_PredictionString(row), axis=1)
submission_df.head().apply(lambda row: update_PredictionString_v2(row), axis=1)
# submission_df.head().apply(lambda row: update_PredictionString_v3(row), axis=1)


In [ ]:
submission_df.tail().apply(lambda row: update_PredictionString_v2(row), axis=1)


In [ ]:
# submission_df['PredictionString'] = submission_df.apply(lambda row: update_PredictionString(row), axis=1)
submission_df['PredictionString'] = submission_df.apply(lambda row: update_PredictionString_v2(row), axis=1)
# submission_df['PredictionString'] = submission_df.apply(lambda row: update_PredictionString_v3(row), axis=1)


In [ ]:
submission_df.head(20)

In [ ]:
submission_df.tail(20)

In [ ]:
submission_df.to_csv('submission.csv', index=False)

### now trigger image binary clf to detect opacity vs. none

https://www.kaggle.com/rbhambri/chexnet-transfer-learning-binary-image-clf



In [ ]:
# 7f090fbd8e7c_image,None 1 0 0 1 1
# 898d24f3cda6_image,opacity 0.479425 17 105 35 171
# a0f6e42396fc_image,opacity 0.37193 171 231 106 182 opacity 0.431259 67 127 61 179
# 422c0e145066_image,None 1 0 0 1 1
# 46719b856de1_image,None 1 0 0 1 1
# 31c07523a69a_image,opacity 0.427264 53 113 78 200 opacity 0.565582 168 238 58 194
# f77d7d1aebab_image,None 1 0 0 1 1
# ccc5b63ca96d_image,None 1 0 0 1 1
# 5e8ac1fe2b82_image,opacity 0.318599 166 232 126 242 opacity 0.427661 48 120 78 234
